In [1]:
from transformers import pipeline

In [ ]:
model_name = "llm-book/bert-base-japanese-v3-jsts"

text_sim_pipeline = pipeline(
    model=model_name,
    function_to_apply="none",
)

text = "ある朝、グレゴール・ザムザが気がかりな夢から目ざめたとき、自分がベッドの上で一匹の巨大な毒虫に変ってしまっているのに気づいた。"
sim_text = "彼は甲殻のように固い背中を下にして横たわり、頭を少し上げると、何本もの弓形のすじにわかれてこんもりと盛り上がっている自分の茶色の腹が見えた。"
result = text_sim_pipeline({"text": text, "text_pair": sim_text})
print(result["score"])

dissim_text = "腹の盛り上がりの上には、かけぶとんがすっかりずり落ちそうになって、まだやっともちこたえていた。"
result = text_sim_pipeline({"text": text, "text_pair": dissim_text})
print(result["score"])

from torch.nn.functional import cosine_similarity

sim_enc_pipeline = pipeline(
    model="llm-book/bert-base-japanese-v3-unsup-simcse-jawiki",
    task="feature-extraction",
)

text_emb = sim_enc_pipeline(text, return_tensors=True)[0][0]
sim_emb = sim_enc_pipeline(sim_text, return_tensors=True)[0][0]
sim_pair_score = cosine_similarity(text_emb, sim_emb, dim=0)
print(sim_pair_score.item())

dissim_emb = sim_enc_pipeline(dissim_text, return_tensors=True)[0][0]
dissim_pair_score = cosine_similarity(text_emb, dissim_emb, dim=0)
print(dissim_pair_score.item())


Device set to use mps:0


1.269738793373108
0.25910577178001404


Device set to use mps:0


0.61335289478302
0.487723708152771


In [15]:
from torch.nn.functional import cosine_similarity

sim_enc_pipeline = pipeline(
    model="llm-book/bert-base-japanese-v3-unsup-simcse-jawiki",
    task="feature-extraction",
)
text_emb = sim_enc_pipeline(text, return_tensors=True)[0][0]
sim_emb = sim_enc_pipeline(sim_text, return_tensors=True)[0][0]
sim_pair_score = cosine_similarity(text_emb, sim_emb, dim=0)
print(sim_pair_score.item())


Device set to use mps:0


0.7066593766212463


In [ ]:
from sentence_transformers import SentenceTransformer

model_name = "hotchpotch/static-embedding-japanese"
model = SentenceTransformer(model_name, device="cpu")

query = "ある朝、グレゴール・ザムザが気がかりな夢から目ざめたとき、自分がベッドの上で一匹の巨大な毒虫に変ってしまっているのに気づいた。"
docs = [
    "彼は甲殻のように固い背中を下にして横たわり、頭を少し上げると、何本もの弓形のすじにわかれてこんもりと盛り上がっている自分の茶色の腹が見えた。",
    "腹の盛り上がりの上には、かけぶとんがすっかりずり落ちそうになって、まだやっともちこたえていた。",
    "ふだんの大きさに比べると情けないくらいかぼそいたくさんの足が自分の眼の前にしょんぼりと光っていた。",
    "「おれはどうしたのだろう？」と、彼は思った。",
]

embeddings = model.encode([query] + docs)
print(embeddings.shape)
similarities = model.similarity(embeddings[0], embeddings[1:])
for i, similarity in enumerate(similarities[0].tolist()):
    print(f"{similarity:.04f}: {docs[i]}")


(5, 1024)
0.1229: 彼は甲殻のように固い背中を下にして横たわり、頭を少し上げると、何本もの弓形のすじにわかれてこんもりと盛り上がっている自分の茶色の腹が見えた。
0.0761: 腹の盛り上がりの上には、かけぶとんがすっかりずり落ちそうになって、まだやっともちこたえていた。
0.1535: ふだんの大きさに比べると情けないくらいかぼそいたくさんの足が自分の眼の前にしょんぼりと光っていた。
0.1271: 「おれはどうしたのだろう？」と、彼は思った。
